# CSX46 - Class 20 - Boolean Networks

In this class, we will implement the *Li et al.* (PNAS, 2004) Boolean network model of the protein network controlling the yeast cell cycle. This model has 12 nodes and an interesting way of implementing state changes (dynamics) that differs slightly from a "canonical" Boolean network. After implementing the model, we will run a bunch of forward simulations using random initial states of the proteins, and we will determine (for each random initial state) which fixed-point (i.e., steady) state the system ends up in. We will compute the size of the "basin" for each fixed point (sometimes called the "attractor basin") as the number of random initial states that ultimately lead to that particular fixed-point state. 

We don't need many packages for this notebook, just a few of the basics:

In [1]:
import itertools
import numpy
import random

We will define a list of "node names" and an adjacency matrix. You will see that the ajacency matrix has some negative entries!  These correspond to inhibitory edges. I have saved you a lot of trouble by typing in the adjacency matrix; I made many mistakes before I got it right!

In [2]:
nodes = ['Cell Size', 
         'Cln3', 
         'MBF', 
         'Clb5,6', 
         'Mcm1/SFF', 
         'Swi5', 
         'Sic1', 
         'Clb1,2', 
         'Cdc20&Cdc14', 
         'Cdh1', 
         'Cln1,2', 
         'SBF']

N = len(nodes)

# define the transition matrix 
a = numpy.zeros([N, N])
a[0,1] = 1
a[1,1] = -1
a[1,2] = 1
a[1,11] = 1
a[2,3] = 1
a[3,4] = 1
a[3,6] = -1
a[3,7] = 1
a[3,9] = -1
a[4,4] = -1
a[4,5] = 1
a[4,7] = 1
a[4,8] = 1
a[5,5] = -1
a[5,6] = 1
a[6,3] = -1
a[6,7] = -1
a[7,2] = -1
a[7,4] = 1
a[7,5] = -1
a[7,6] = -1
a[7,8] = 1
a[7,9] = -1
a[7,11] = -1
a[8,3] = -1
a[8,5] = 1
a[8,6] = 1
a[8,7] = -1
a[8,8] = -1
a[8,9] = 1
a[9,7] = -1
a[10,6] = -1
a[10,9] = -1
a[10,10] = -1
a[11,10] = 1
a = numpy.matrix(a)

Let's take a look at the matrix

In [3]:
a

matrix([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1.,  1.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  1., -1., -1.,  0.,  1., -1.,  0., -1.],
        [ 0.,  0.,  0., -1.,  0.,  1.,  1., -1., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

The Li *et al.* article describes seven "fixed points" for their Boolean network model. They can be represented as a 7x12 matrix, as shown here:

In [4]:
# define the matrix of states for the fixed points
num_fp = 7
fixed_points = numpy.zeros([num_fp, N])
fixed_points[0, 6] = 1
fixed_points[0, 9] = 1
fixed_points[1, 10] = 1
fixed_points[1, 11] = 1
fixed_points[2, 2] = 1
fixed_points[2, 6] = 1
fixed_points[2, 9] = 1
fixed_points[3, 6] = 1
fixed_points[4, 2] = 1
fixed_points[4, 6] = 1
fixed_points[6, 9] = 1
fixed_points = numpy.matrix(fixed_points)

Let's take a look at the fixed-point matrix; each row is a Boolean "state vector" for the 12 nodes

In [5]:
fixed_points

matrix([[0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

Define a function `hamming.dist` that gives the hamming distance between two states of the Boolean network (as numpy arrays of ones and zeroes)

In [6]:
def hamming_dist(x1, x2):
    return numpy.sum(numpy.abs(x1-x2))

Define a function `evolve` that takes the network from one Boolean vector state to another Boolean vector state

In [7]:
def evolve(state):
    result = numpy.array(a.transpose().dot(state))
    result = numpy.reshape(result, N)
    result[result > 0] = 1
    result[result == 0] = state[result == 0]
    result[result < 0] = 0
    return result

Write a function that runs 10,000 simulations of the network. In each simulation, the procedure is:
- create a random binary vector of length 12, and call that vector `state` (make sure the zeroth element is set to zero)
- iteratively call "evolve", passing the `state` to evolve and then updating `state` with the return value from `evolve`
- check if `state` changes in the last call to `evolve`; if it does not, then you have reached a fixed point; stop iterating
- compare the state to the rows of `fixed_points`; for the unique row `j` for which you find a match, add `j` to the list `basin_ids`.
- use `numpy.bincount` to count the number of times each basin ID appears in `basin_ids`

In [8]:
basin_ids = []
for _ in itertools.repeat(None, 10000):
    state = [0]
    for pos in range(0, (N-1)):
        state.append(random.randint(0,1))
    state = numpy.array(state)
    state_new = numpy.array([-1]*N)
    while(True):
        state_new = evolve(state)
        if hamming_dist(state, state_new) == 0:
            break
        state = state_new
    for j in range(0, num_fp):
        fp_state = numpy.array(fixed_points[j,])
        fp_state = numpy.reshape(fp_state, N)
        if hamming_dist(state, fp_state) == 0:
            basin_ids.append(j)

In [9]:
fp_counts = numpy.bincount(basin_ids)
fp_counts

array([8638,  714,  527,   41,   34,   37,    9])

Which fixed-point has the highest count?  Compare to Table 1 in Li *et al.* 2004.

In [10]:
{nodes[i]: fixed_points[numpy.argmax(fp_counts),].tolist()[0][i] for i in range(0,12)}

{'Cell Size': 0.0,
 'Cln3': 0.0,
 'MBF': 0.0,
 'Clb5,6': 0.0,
 'Mcm1/SFF': 0.0,
 'Swi5': 0.0,
 'Sic1': 1.0,
 'Clb1,2': 0.0,
 'Cdc20&Cdc14': 0.0,
 'Cdh1': 1.0,
 'Cln1,2': 0.0,
 'SBF': 0.0}